#가설2: 심리적 압박·유혹·과장 표현이 많은 매물은 전세사기일 확률이 높을 것이다.
텍스트 기반 이상치 점수 text_anomaly_score 생성

In [1]:
# =========================
# 0. 라이브러리 설치
# =========================
!pip install -q sentence-transformers scikit-learn tqdm

In [2]:
# =========================
# 1. 기본 import + 구글 드라이브 마운트
# =========================
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import IsolationForest

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# =========================
# 2. CSV 로드
#    (이미 image_anomaly_score 가 붙어있는 파일 사용)
# =========================
csv_path = "/content/drive/MyDrive/daangn_list_detail_with_image_score.csv"  # 경로는 상황에 맞게 수정
df = pd.read_csv(csv_path)

print("데이터 shape:", df.shape)
print(df.columns)

# description 컬럼이 있는지 확인 (이름이 다르면 여기 수정!)
assert "description" in df.columns, "description 컬럼 이름을 확인해 주세요."

데이터 shape: (1059, 17)
Index(['area', 'identifier', 'description', 'image_count', 'image',
       'building_name', 'building_usage', 'exclusive_area', 'floor',
       'direction', 'maintenance_fee', 'built_year', 'total_floor', 'price',
       'address', 'register_date', 'image_anomaly_score'],
      dtype='object')


In [4]:
# =========================
# 3. 텍스트 전처리
# =========================
# NaN -> 빈 문자열, 타입 통일
df["description"] = df["description"].fillna("").astype(str).str.strip()

# 너무 짧은 설명(정보 부족) 마스크: 나중에 패널티 줄 때 사용
short_desc_mask = df["description"].str.len() < 20  # 기준 길이는 자유롭게 조정 가능
print("짧은 설명(20자 미만) 개수:", short_desc_mask.sum())

texts = df["description"].tolist()

짧은 설명(20자 미만) 개수: 23


In [5]:
# =========================
# 4. Sentence-BERT 로 임베딩 추출
# =========================
# 한국어 SBERT 모델 로드 (인터넷 필요)
model_name = "jhgan/ko-sbert-multitask"  # 또는 "sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens"
model = SentenceTransformer(model_name)

# 문장 -> 벡터 (임베딩)
# show_progress_bar=True 로 진행상황 확인 가능
embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)
print("임베딩 shape:", embeddings.shape)  # (N, 768 등)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

임베딩 shape: (1059, 768)


In [6]:
# =========================
# 5. IsolationForest 로 비지도 이상탐지
# =========================
iso = IsolationForest(
    n_estimators=200,
    contamination=0.05,   # 상위 5% 정도를 이상치로 간주 (튜닝 가능)
    random_state=42,
    n_jobs=-1
)
iso.fit(embeddings)

# score_samples: 값이 클수록 "정상", 작을수록 "이상"
raw_scores = -iso.score_samples(embeddings)  # 부호 반전 → 값이 클수록 "이상"

print("raw_scores 예시:", raw_scores[:5])


raw_scores 예시: [0.4477638  0.41362421 0.4244173  0.43376888 0.41700253]


In [7]:
# =========================
# 6. 0.0 ~ 1.0 범위로 min-max 정규화 → text_anomaly_score
# =========================
min_s = raw_scores.min()
max_s = raw_scores.max()
denom = max_s - min_s

if denom < 1e-8:
    # 점수가 거의 모두 같은 극단적인 경우 → 전부 0으로 처리
    norm_scores = np.zeros_like(raw_scores)
else:
    norm_scores = (raw_scores - min_s) / (denom + 1e-9)

df["text_anomaly_score"] = norm_scores

print("정규화 전/후 최소, 최대:")
print("raw:", min_s, max_s)
print("norm:", df["text_anomaly_score"].min(), df["text_anomaly_score"].max())

정규화 전/후 최소, 최대:
raw: 0.38143157011795953 0.5807525448623524
norm: 0.0 0.9999999949829665


In [9]:
# =========================
# 7. 설명이 거의 없는 매물에 대한 패널티 (선택적)
#    - 너무 짧은 description 은 정보가 부족하므로, 텍스트 관점에서 좀 더 의심스럽다고 봄
# =========================
penalty = 0.2  # 얼마나 올릴지 (0 ~ 1 사이, 상황에 따라 조정)
df.loc[short_desc_mask, "text_anomaly_score"] = np.clip(
    df.loc[short_desc_mask, "text_anomaly_score"] + penalty,
    0.0,
    1.0
)

print("패널티 적용 후 text_anomaly_score 요약:")
print(df["text_anomaly_score"].describe())

패널티 적용 후 text_anomaly_score 요약:
count    1059.000000
mean        0.250930
std         0.178662
min         0.000000
25%         0.133269
50%         0.205460
75%         0.307640
max         1.000000
Name: text_anomaly_score, dtype: float64


In [10]:
# =========================
# 8. CSV 저장 (image_anomaly_score + text_anomaly_score 포함)
# =========================
out_path = "/content/drive/MyDrive/daangn_list_detail_with_image_text_score.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("저장 완료:", out_path)

저장 완료: /content/drive/MyDrive/daangn_list_detail_with_image_text_score.csv


daangn_list_detail_with_image_score.csv 를 읽고

description 텍스트를 SBERT로 벡터화하고

IsolationForest로 “텍스트 패턴이 얼마나 튀는지”를 수치화해서

0~1 사이의 text_anomaly_score를 만든 뒤

짧은 설명에는 약간의 패널티를 더하고

daangn_list_detail_with_image_text_score.csv 로 저장하는 코드